In [2]:
! pip install -U spacy
! pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)
     |████████████████████████████████| 778.7MB 22kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-3.0.0-cp37-none-any.whl size=778750482 sha256=4b6449a05cf87b15a668dfc5b12f3487c045e37bbf927bf793b44ccc434845ea
  Stored in directory: /root/.cache/pip/wheels/76/d4/24/e2bbc81bef966db10207f8280a6cdfd45d7a1d2f97657ab1a5
Successfully built en-core-web-lg


In [3]:
import os
import pprint
import string
import re
import pandas as pd
import numpy as np
import plotly.express as px

import nltk
from nltk.text import Text
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
import spacy
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, AdaBoostClassifier

from sklearn.metrics.pairwise import cosine_similarity

nltk.download('all')
pp = pprint.PrettyPrinter(indent=4, compact=True)
nlp = spacy.load("en_core_web_lg")

english_stops = set(stopwords.words('english'))
english_stops.update(string.punctuation)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [4]:
# Activacndo Google Drive para guardar la información
from google.colab import drive
drive.mount('/content/gdrive')
df_path = '/content/gdrive/My Drive/Facultad/Austral/Maestria 1/13 - Text mining/00 - TPs/Web Scraping/TED_Talk.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
tedx_df = pd.read_csv(df_path)
tedx_df.head()

,talk__id,talk__name,talk__description,view_count,duration,transcript,video_type_name,event,speaker__id,speaker__name,speaker__description,speaker__who_he_is,speaker__why_listen,all_speakers_details,recording_date,published_timestamp,talks__tags,number_of__tags,language,native_language,url__webpage,talk__more_resources,number_of__talk__more_resources,talk__recommendations__blurb,talk__recommendations,number_of__talk__recommendations,related_talks,number_of__related_talks
0,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70658143,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,"['creativity', 'culture', 'dance', 'education'...",7,en,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6
1,2405,This is what happens when you reply to spam email,"Suspicious emails: unclaimed insurance bonds, ...",63741646,588,"A few years ago, I got one of those spam email...",TED Stage Talk,TEDGlobal>Geneva,2664.0,James Veitch,"Comedian, writer","For James Veitch, spam emails proved the perfe...",James Veitch's correspondence with email spamm...,"[{'id': '2664', 'slug': 'james_veitch', 'is_pu...",2015-12-08,2020-10-23 13:51:31,"['comedy', 'curiosity', 'communication', 'humo...",5,en,en,https://www.ted.com/talks/james_veitch_this_is...,[],0,NaN,NaN,0,"[{'id': '2236', 'hero': 'https://pe.tedcdn.com...",4
2,1569,Your body language may shape who you are,(NOTE: Some of the findings presented in this ...,61580489,1262,So I want to start by offering you a free no-t...,TED Stage Talk,TEDGlobal 2012,1284.0,Amy Cuddy,Social psychologist,Amy Cuddy’s research on body language reveals ...,Amy Cuddy wasn’t supposed to become a successf...,"[{'id': '1284', 'slug': 'amy_cuddy', 'is_publi...",2012-06-26,2012-10-01 15:00:08,"['body language', 'brain', 'business', 'psycho...",7,en,en,https://www.ted.com/talks/amy_cuddy_your_body_...,"[{'status': 'approved', 'headline': 'Presence:...",4,NaN,NaN,0,"[{'id': '1042', 'hero': 'https://s3.amazonaws....",6
3,848,How great leaders inspire action,Simon Sinek has a simple but powerful model fo...,54989014,1084,How do you explain when things don't go as we ...,TEDx Talk,TEDxPuget Sound,703.0,Simon Sinek,Leadership expert,Simon Sinek explores how leaders can inspire c...,Fascinated by the leaders who make impact in t...,"[{'id': '703', 'slug': 'simon_sinek', 'is_publ...",2009-09-17,2010-05-04 09:31:00,"['TEDx', 'business', 'entrepreneur', 'leadersh...",5,en,en,https://www.ted.com/talks/simon_sinek_how_grea...,"[{'status': 'approved', 'headline': '*Start wi...",1,The leadership expert shares books and films t...,"[{'title': 'Reading List', 'description': '', ...",1,"[{'id': '1998', 'hero': 'https://pe.tedcdn.com...",6
4,1042,The power of vulnerability,Brené Brown studies human connection -- our ab...,53547088,1219,"So, I'll start with this: a couple years ago, ...",TEDx Talk,TEDxHouston,914.0,Brené Brown,"Vulnerability researcher, storyteller","Dr. Brené Brown studies courage, vulnerability...",Brené Brown is a research professor at the Uni...,"[{'id': '914', 'slug': 'brene_brown', 'is_publ...",2010-06-06,2010-12-23 14:45:00,"['TEDx', 'communication', 'culture', 'depressi...",10,en,en,https://www.ted.com/talks/brene_brown_the_powe...,"[{'status': 'approved', 'headline': '*Daring G...",2,Dive deeper into shame and vulnerability (in a...,"[{'title': 'Reading List', 'description': '', ...",1,"[{'id': '1391', 'hero': 'https://s3.amazonaws....",6


In [6]:
def remove_stop_words(text):
  """
    Remueve stop words en inglés

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar

    Returns
    -------
    list
      lista de palabras sin los stop words
  """
  return [token for token in text if token.lower() not in english_stops]

In [7]:
def lematize_words(text):
  """
    Lematización de palabras - aplica lematización de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre los cuales se aplicará la lematización
    
    Returns
    -------
    list
      lista con todas las lematizaciones de las palabras
  """
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [8]:
def remove_meaningless_words(text):
  """
    Remueve palabras sin significado

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar
    
    Returns
    -------
    list
      lista de palabras filtrada en base a expresiones regulares
  """
  patterns = [r"(^={1,}=$)", r'\u200b']
  tokens = text
  for pattern in patterns:
    regexp = re.compile(pattern)
    tokens = [token for token in tokens if not regexp.search(token)]
  return tokens

In [9]:
def clean_short_words(text):
  """
    Limpia palabras con longitud 1

    Attributes
    ----------
    text: str
      documento a tokenizar
    
    Returns
    -------
    list
      lista de tokens
  """
  return [word for word in text if len(word) > 1]

In [10]:
def tokenize(text, mode='word'): 
  """
    Tokenización de documento - tokeniza un documento por palabra o por oración

    Attributes
    ----------
    text: str
      documento a tokenizar
    mode: str, optional
      método de tokenización (default: 'word' (por palabra))
    
    Returns
    -------
    list
      lista de tokens 
    
    Raises
    ------
      Exception
        si el mode no es 'word' o 'sentence'
  """
  if mode == 'word':
    return word_tokenize(text, language='english')
  elif mode == 'sentence':
    return sent_tokenize(text, language='english')
  else:
    raise Exception('metodo de tokenizacion no encontrado')

In [11]:
def similarity_btw_docs(matrix):
  """
    Similitud entre documentos - calcula la similitud entre documentos utilizando Similitud del Coseno

    Attributes
    ----------
    matrix: scipy matrix
      Matriz dispersa para calcular la similaridad

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con el grado de similaridad entre documentos (de 0 a 1)
  """
  matrix_simil = cosine_similarity(matrix)
  return pd.DataFrame(matrix_simil)

In [12]:
def pre_procesamiento_texto(text):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    text: str
      documento a analizar

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con las 20 palabras que más se repiten y su frecuencia
  """
  tokenized = tokenize(text)
  without_stops = remove_stop_words(tokenized)
  meaningfull_tokens = remove_meaningless_words(without_stops)
  without_short_words = clean_short_words(meaningfull_tokens)
  lematized_words = lematize_words(without_short_words)
  return lematized_words

In [13]:
def remove_character(serie, char):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    serie: pd.Serie
      columna de dataframe a modificar
    char: char
      caracter a remover

    Returns
    -------
    pd.Serie
      retorna una serie
  """
  return serie.str.replace(char, '')

In [14]:
tedx_df = tedx_df.assign(talks__tags=tedx_df['talks__tags'].str.split(',')).explode('talks__tags')
tedx_df['tag'] = remove_character(remove_character(remove_character(tedx_df['talks__tags'], "["), "'"), "]")
#tedx_df['tag'] = tedx_df['talks__tags'].str.replace("[", "").str.replace("'", "").str.replace("]", "")
del tedx_df['talks__tags']

In [15]:
tedx_df.head()

,talk__id,talk__name,talk__description,view_count,duration,transcript,video_type_name,event,speaker__id,speaker__name,speaker__description,speaker__who_he_is,speaker__why_listen,all_speakers_details,recording_date,published_timestamp,number_of__tags,language,native_language,url__webpage,talk__more_resources,number_of__talk__more_resources,talk__recommendations__blurb,talk__recommendations,number_of__talk__recommendations,related_talks,number_of__related_talks,tag
0,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70658143,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,7,en,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,creativity
0,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70658143,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,7,en,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,culture
0,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70658143,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,7,en,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,dance
0,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70658143,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,7,en,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,education
0,66,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,70658143,1164,Good morning. How are you?(Audience) Good.It's...,TED Stage Talk,TED2006,69.0,Sir Ken Robinson,"Author, educator",Creativity expert Sir Ken Robinson challenged ...,Why don't we get the best out of people? Sir K...,"[{'id': '69', 'slug': 'sir_ken_robinson', 'is_...",2006-02-25,2006-06-27 00:11:00,7,en,en,https://www.ted.com/talks/sir_ken_robinson_do_...,"[{'status': 'approved', 'headline': '*You, You...",3,The education expert has a lot to say about th...,"[{'title': 'Reading list', 'description': '', ...",1,"[{'id': '30217', 'hero': 'https://s3.amazonaws...",6,parenting


In [16]:
tedx_df = tedx_df[tedx_df['transcript'] != np.nan]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(tedx_df.transcript, tedx_df.tag, test_size=.30)
X_train = X_train.values.astype('U')
X_test = X_test.values.astype('U')

In [18]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=pre_procesamiento_texto)

In [19]:
estimators = [
    ('rf', RandomForestClassifier()),
    ('svr', LinearSVC()),
    ('nb', GaussianNB()),
    ('dt', DecisionTreeClassifier()),
    ('ab', AdaBoostClassifier())
]

model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [20]:
ml_pipeline = Pipeline(steps=[
                              ('preprocessor', tfidf_vectorizer),
                              ('classifier', model)
])

In [ ]:
ml_pipeline.fit(X_train, y_train)

In [ ]:
# TODO: Aplicar la función de similarity_btw_docs